### Задание 1.
Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import time
import datetime
%matplotlib inline

df = pd.read_csv("features.csv", index_col='match_id')
df.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


#### Описание признаков в таблице

- `match_id`: идентификатор матча в наборе данных
- `start_time`: время начала матча (unixtime)
- `lobby_type`: тип комнаты, в которой собираются игроки (расшифровка в `dictionaries/lobbies.csv`)
- Наборы признаков для каждого игрока (игроки команды Radiant — префикс `rN`, Dire — `dN`):
    - `r1_hero`: герой игрока (расшифровка в dictionaries/heroes.csv)
    - `r1_level`: максимальный достигнутый уровень героя (за первые 5 игровых минут)
    - `r1_xp`: максимальный полученный опыт
    - `r1_gold`: достигнутая ценность героя
    - `r1_lh`: число убитых юнитов
    - `r1_kills`: число убитых игроков
    - `r1_deaths`: число смертей героя
    - `r1_items`: число купленных предметов
- Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
    - `first_blood_time`: игровое время первой крови
    - `first_blood_team`: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
    - `first_blood_player1`: игрок, причастный к событию
    - `first_blood_player2`: второй игрок, причастный к событию
- Признаки для каждой команды (префиксы `radiant_` и `dire_`)
    - `radiant_bottle_time`: время первого приобретения командой предмета "bottle"
    - `radiant_courier_time`: время приобретения предмета "courier" 
    - `radiant_flying_courier_time`: время приобретения предмета "flying_courier" 
    - `radiant_tpscroll_count`: число предметов "tpscroll" за первые 5 минут
    - `radiant_boots_count`: число предметов "boots"
    - `radiant_ward_observer_count`: число предметов "ward_observer"
    - `radiant_ward_sentry_count`: число предметов "ward_sentry"
    - `radiant_first_ward_time`: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
- Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
    - `duration`: длительность
    - `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    - Состояние башен и барраков к концу матча (см. описание полей набора данных)
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`

In [2]:
features_to_delete = ["duration", "radiant_win", "tower_status_radiant", "tower_status_dire",
                      "barracks_status_radiant", "barracks_status_dire"]
features = df.drop(features_to_delete, axis=1)
y = df["radiant_win"]

features[:5]

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35,103,-84,221,3,4,2,2,-52
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20,149,-84,195,5,4,3,1,-5
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39,45,-77,221,3,4,3,1,13
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30,124,-80,184,0,4,2,0,27
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46,182,-80,225,6,3,3,0,-16


### Задание 2.
Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [4]:
features.shape

(97230, 102)

In [5]:
print "Number of features with missed vales", sum(features.count()<97230)
features[features.columns[features.count()<97230]].count()

Number of features with missed vales 12


first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

#### Ответ 2.
Всего 12 признаков, имеющие пропущенные значения:
- `first_blood_time`
- `first_blood_team`
- `first_blood_player1`
- `first_blood_player2`
- `radiant_bottle_time`
- `radiant_courier_time`
- `radiant_flying_courier_time`
- `radiant_first_ward_time`
- `dire_bottle_time`
- `dire_courier_time`
- `dire_flying_courier_time`
- `dire_first_ward_time`

Первые три признака (`first_blood_time`, `first_blood_team`, `first_blood_player1`) рассказывают про парметры первой крови в битве и, как сообщается в описании: "Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение", то есть в 77677 играх из 97230 в тренировочном сете это событие прозошло позже или не произошло.
Также видно, что не в каждой игре был приобретен предмет "bottle" (метрики `radiant_bottle_time` и `dire_bottle_time`).

### Задание 3.
Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [6]:
X = features.fillna(0)
sum(X.count() < 97230)

0

In [7]:
features_avg = features
for f in features.columns[features.count()<97230]:
    features_avg[f] = features[f].fillna(features[f].median())  # replace missed values of features with median value for
sum(features_avg.count()<97230)

0

### Задание 4.
Какой столбец содержит целевую переменную? Запишите его название.

#### Ответ 4.
Столбец `radiant_win`, теперь обозначен переменной y.

### Задание 5.
Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [8]:
df_shuffle = pd.concat([X,y], axis=1).reindex(np.random.permutation(pd.concat([X,y], axis=1).index))
X_shuffled = df_shuffle.drop("radiant_win", axis=1)
y_shuffled = df_shuffle["radiant_win"]
X_part, y_part = X_shuffled[:len(y_shuffled)/3], y_shuffled[:len(y_shuffled)/3]

In [188]:
for n_estimators in [10, 20, 30, 40, 50, 60, 70, 80]:
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_estimators, random_state=42)
    kf = KFold(X_part.shape[0], random_state=42, shuffle=True, n_folds=5)
    print n_estimators, cross_val_score(clf, X_part, y_part, cv=kf, scoring='roc_auc').mean()
    print 'Time elapsed:', datetime.datetime.now() - start_time

10 0.661546144523
Time elapsed: 0:00:20.971732
20 0.678170289854
Time elapsed: 0:00:38.975785
30 0.685511390559
Time elapsed: 0:00:56.840354
40 0.690535291062
Time elapsed: 0:01:09.077508
50 0.69334395375
Time elapsed: 0:01:39.517367
60 0.696006645405
Time elapsed: 0:01:55.506125
70 0.697403793977
Time elapsed: 0:02:16.837866
80 0.699109748719
Time elapsed: 0:02:37.452757


In [189]:
# There is a trade-off between learning_rate and n_estimators. Let's try different learning_rate meanings:

for learning_rate in [1, 0.5, 0.3, 0.2, 0.1]:
    clf = GradientBoostingClassifier(n_estimators=30, random_state=42, learning_rate=learning_rate)
    kf = KFold(X_part.shape[0], random_state=42, shuffle=True, n_folds=5)
    print learning_rate, cross_val_score(clf, X_part, y_part, cv=kf, scoring='roc_auc').mean()

1 0.677939334086
0.5 0.695153288756
0.3 0.696324353773
0.2 0.692981839394
0.1 0.685511390559


In [190]:
clf = GradientBoostingClassifier(n_estimators=50, learning_rate=0.3)
kf = KFold(X.shape[0], random_state=42, shuffle=True, n_folds=5)
cross_val_score(clf, X, y, cv=kf, scoring='roc_auc').mean()

0.70010992092357505

#### Ответ 5.
Поскольку настройка алгоритмов занимала слишком много времени, я сначала попробовала вычисления на трети выборки. Как видно, время для настройки классификатора с количеством деревьев от 10 до 80 время варьируется от 20 секунд до 2,5 минут. Помня домашнее задание по теме градиентного бустинга, ожидаю похожую ситуацию: ошибки на тестовом сете после некоторого уменьшения выйдут на уровень константы.

### Задание 2.1.
Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

X_scal = StandardScaler().fit_transform(X)

for C in [10, 5, 1, 0.5, 0.2, 0.1, 0.05, 0.01]:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', C=C, random_state=42)
    kf = KFold(X_scal.shape[0], random_state=42, shuffle=True, n_folds=5)
    print C, cross_val_score(clf, X_scal, y, cv=kf, scoring='roc_auc').mean()
    
    print 'Time elapsed:', datetime.datetime.now() - start_time

10 0.716522287837
Time elapsed: 0:00:16.217781
5

KeyboardInterrupt: 

#### Ответ 2.1
Наилучшее значение score получилось ~0.716, практически не зависит от параметра C. Это значение получилось лучше чем при градиентном бустинге (0.70).
При этом логистическая регрессия работает значительно быстрее градиентного бустинга: до 20 секунд.

### Задание 2.2.
Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [10]:
params_to_drop = ["lobby_type", "r1_hero", "r2_hero", "r3_hero", "r4_hero", "r5_hero",
                  "d1_hero", "d2_hero", "d3_hero", "d4_hero", "d5_hero"]

X_wo_features = X.drop(params_to_drop, axis=1)
X_cutted = StandardScaler().fit_transform(X_wo_features)
for C in [10, 5, 1, 0.5, 0.2, 0.1, 0.05, 0.01]:
    clf = LogisticRegression(penalty='l2', C=C, random_state=42)
    kf = KFold(X_cutted.shape[0], random_state=42, shuffle=True, n_folds=5)
    print C, cross_val_score(clf, X_cutted, y, cv=kf, scoring='roc_auc').mean()

 10 0.716529971503
5 0.716530134606
1

KeyboardInterrupt: 

#### Ответ 2.2.
При изъятии указанных выше параметров улучшение качества модели почти не заметно, это может свидетельствовать о том, что в данном виде (в виде числовых признаков) эти параметры не играют важной роли в построении модели. Пример: герой под номером 1 очень похож на героя №2 и сильно не похож на героя №50, что может не соовтетствоать действительности: герои не ранжированы в этом списке.

### Задание 2.3.
На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [11]:
len(pd.Series(X[[i for i in params_to_drop[1:]]].values.ravel()).unique())

108

#### Ответ 2.3.
108 различных героев.

### Задание 2.4.
Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [12]:
N = 108

data = X[params_to_drop[1:]]
X_pick = np.zeros((data.shape[0], N+4))

for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1.0
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1.0
X_pick_df = pd.DataFrame(X_pick, columns=["hero_"+str(i) for i in range(X_pick.shape[1])], index=X_wo_features.index)

X_categor = pd.concat([X_pick_df, X_wo_features], axis=1)

In [13]:
X_categor[:4]

,hero_0,hero_1,hero_2,hero_3,hero_4,hero_5,hero_6,hero_7,hero_8,hero_9,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,-1,0,0,0,0,0,0,...,0,35,103,-84,221,3,4,2,2,-52
1,0,0,0,0,0,0,-1,0,0,0,...,0,-20,149,-84,195,5,4,3,1,-5
2,0,0,0,1,0,0,0,0,0,0,...,1,-39,45,-77,221,3,4,3,1,13
3,0,0,0,0,0,0,0,0,0,0,...,0,-30,124,-80,184,0,4,2,0,27


### Задание 2.5.
Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [14]:
X_categor_scaled = StandardScaler().fit_transform(X_categor)
for C in [10, 5, 1, 0.5, 0.2, 0.1, 0.05, 0.01]:
    clf = LogisticRegression(penalty='l2', C=C, random_state=42)
    kf = KFold(X_categor_scaled.shape[0], random_state=42, shuffle=True, n_folds=5)
    print C, cross_val_score(clf, X_categor_scaled, y, cv=kf, scoring='roc_auc').mean()

10 0.751924753045
5 0.751924771047
1 0.751925108955
0.5 0.751925750822
0.2 0.751927166982
0.1 0.75193003332
0.05 0.751936273132
0.01 0.751964042103


In [17]:
# try sklearn.ensemble.BaggingClassifier
from sklearn.ensemble import BaggingClassifier
clf = LogisticRegression(penalty='l2', C=0.05, random_state=42)

for n in [5,7,10,15]:
    clf_bagging = BaggingClassifier(base_estimator=clf, n_estimators=n)
    kf = KFold(X_categor_scaled.shape[0], random_state=42, shuffle=True, n_folds=5)
    print n, cross_val_score(clf, X_categor_scaled, y, cv=kf, scoring='roc_auc').mean()

5 0.751936273132
7 0.751936273132
10 0.751936273132
15 0.751936273132


#### Ответ 2.5.
Качество получилось лучше - 0.752. Использование dummy variables позволило представить фичи "выбранные герои" в качестве категориальных.

### Задание 2.6.
Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [15]:
test = pd.read_csv("features_test.csv", index_col='match_id')
test[:5]

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12,247,-86,272,3,4,2,0,118
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29,168,-54,NaN,3,2,2,1,16
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22,46,-87,186,1,3,3,0,-34
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49,30,-89,210,3,4,2,1,-26
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36,180,-86,180,1,3,2,1,-33


In [73]:
clf = LogisticRegression(penalty='l2', C=0.05, random_state=42)
clf.fit(X_categor_scaled, y)
test = test.fillna(0)
data_test = test[params_to_drop[1:]]
X_pick_test = np.zeros((data_test.shape[0], N+4))

for i, match_id in enumerate(data_test.index):
    for p in xrange(5):
        X_pick_test[i, data_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1.0
        X_pick_test[i, data_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1.0
X_pick_df_test = pd.DataFrame(X_pick_test, columns=["hero_player"+str(i) for i in range(X_pick_test.shape[1])],
                              index=test.index)

X_categor_test = pd.concat([X_pick_df_test, test.drop(params_to_drop, axis=1)], axis=1)


X_categor_scaled_test = StandardScaler().fit_transform(X_categor_test)
clf.predict_proba(X_categor_scaled_test)[:,1]

array([ 0.82452076,  0.75801149,  0.18731349, ...,  0.23168719,
        0.6317722 ,  0.42661708])

In [85]:
#test_out = pd.concat([pd.DataFrame(X_categor_test.index), pd.DataFrame(clf.predict_proba(X_categor_scaled_test)[:,1])], axis=1)
test_out = pd.DataFrame(clf.predict_proba(X_categor_scaled_test)[:,1], index=X_categor_test.index, columns=['radiant_win'])
test_out

,radiant_win
match_id,
6,0.824521
7,0.758011
10,0.187313
13,0.858859
16,0.238372
18,0.379699
19,0.529730
24,0.564197
33,0.212841


In [86]:
test_out.to_csv('submission.csv')

In [66]:
col_num = X_categor_scaled_test.shape[1]
test_out = features_test.drop(features_test.columns[[range(0, col_num - 1)]], axis=1)
test_out.to_csv('submission.csv') # Kaggle score: 0.75529

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

Меня очень смущают такие вероятности оценки принадлежности к классам, но найти ошибку я не могу =(
Поэтому попробую описать возможные причины.
Это означает, что классификатор построил разделяющую гиперплоскость вообще в стороне от всех точек. Посколько на тренировочном сете на кроссвалидации классификатор показывал разумные (не 0-1, отличающиеся между собой, воспроизводимые) значения, то можно предположить, что дело в преобразовании тестовой выборки.